In [616]:
import pandas as pd
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

n=train.shape[1]-2 #number of features
x_train=train.iloc[:,:n]
y1_train=train.iloc[:,n:n+1] #family
y2_train=train.iloc[:,n+1:n+2] #species

x_test=test.iloc[:,:n] 
y1_test=test.iloc[:,n:n+1] #family
y2_test=test.iloc[:,n+1:n+2] #species

print(n)
# print(train.head())
# print(test.shape)


22


In [617]:
def classify(y):
    x=pd.Series()

    id=y.columns[0]
    idx=y.value_counts().index

    for i in range(len(idx)):
        x[i]=x_train[y[id].str.contains(idx[i][0],case=False,na=False)] #split the data into different classes

    return x


In [618]:
def mapp(y_train,y_test):
    
    f=y_train.value_counts() #family
    # print(f)
    # f.index[0][0]
    mapping = {
        f.index[i][0]: i for i in range(len(f))
    }

    return y_test.replace(mapping) #mapping y_test to 0,1,2,3


In [619]:
def svm(i,x1:pd.DataFrame,j,x2:pd.DataFrame,x_test:pd.DataFrame,g:np.array,r:float) -> int|np.ndarray:
    x=pd.concat([x1,x2]) #combine two classes
    y=pd.DataFrame(np.zeros((x1.shape[0]+x2.shape[0],1)))
    y.iloc[:x1.shape[0]]=1 #label for class 1
    y.iloc[x1.shape[0]:]=-1 #label for class 2

    x=x.values #convert to np
    y=y.values
    x_test=x_test.values

    a=cp.Variable((n,1))
    b=cp.Variable()
    eta=cp.Variable((x.shape[0],1))
    gamma=cp.Parameter(nonneg=True)

    obj=cp.Minimize(cp.norm(a,2)+gamma*cp.norm(eta,1)/x.shape[0])
    const=[cp.multiply(y,cp.matmul(x,a)-b)>=1-eta,eta>=0]

    prob=cp.Problem(obj,const)

    train_error=np.zeros(len(g))
    pred=np.zeros((len(g),len(x_test))) #prediction for test data

    # A=np.zeros((len(g),n,1))
    # B=np.zeros(len(g))
    # test_error=np.zeros(len(g))

    for t in range(len(g)):
        gamma.value=g[t]

        prob.solve()
        
        a1=a.value #pd to np
        b1=b.value
        eta1=eta.value

        # if (i==0 and j==2 and t==2):
        #     # print(a1.T)
        #     print(b1)

        a1[np.abs(a1)<=r]=0 #remove small values
        # print(np.sum(a1==0))

        # print(i,j,t)
        # if (i==0 and j==2 and t==2):
        #     print(a1.T)
        #     print(b1)

        train_error[t]=np.mean(eta1>1e-3) #error rate
        
        pred[t]=np.array([i if ((x_test@a1)[k]-b1)>0 else j for k in range(len(x_test))]) #prediction for test data
    
    train_error1=train_error[train_error>0] #remove gamma with error rate less than 0.01
    pred1=pred[train_error>0]

    if train_error1.size==0: #if all gamma have error rate less than 0.01
        min_train_error=1 
        min_pred=pred[0]
    else:    
        min_train_error=np.min(train_error1) #minimum train error over gamma
        min_pred=pred1[np.argmin(train_error1)] #prediction for test data with minimum train error

    return min_train_error,min_pred


In [620]:
a=np.ones((5,10))
b=np.ones((10,1))
np.dot(a,b).shape

(5, 1)

In [621]:
def rbf_kernel_matrix(X_new, X_sv, sigma, alpha, y):

    X_new_sq = np.sum(X_new**2, axis=1).reshape(-1, 1)  # Shape: (n_samples, 1)
    X_sv_sq = np.sum(X_sv**2, axis=1).reshape(1, -1)   # Shape: (1, n_sv)
    dist = X_new_sq + X_sv_sq - 2 * np.dot(X_new, X_sv.T)  # Shape: (n_samples, n_sv)
    K=np.exp(-dist/(2*sigma**2))

    a=np.sum(np.dot(K, alpha * y),axis=1)

    return a

In [622]:
def svm_kernel(i,x1:pd.DataFrame,j,x2:pd.DataFrame,x_test:pd.DataFrame,r:float,sigma:float,C:float) -> int|np.ndarray:
    x=pd.concat([x1,x2]) #combine two classes
    y=pd.DataFrame(np.zeros((x1.shape[0]+x2.shape[0],1)))
    y.iloc[:x1.shape[0]]=1 #label for class 1
    y.iloc[x1.shape[0]:]=-1 #label for class 2

    x=x.values #convert to np
    y=y.values
    x_test=x_test.values

    n=x.shape[0] #number of data points

    x_square = np.sum(x**2, axis=1, keepdims=True)
    dist = x_square + x_square.T - 2 * x@x.T
    K = np.exp(-0.5 * dist/(sigma**2))

    Y = np.diag(y.reshape(-1)) #diagonal matrix of y
    Q = Y@K@Y

    alpha = cp.Variable(n)
    c = cp.Parameter(nonneg=True)
    
    obj = cp.Minimize(0.5 * cp.quad_form(alpha, Q) - cp.sum(alpha)) #dual obj
    const = [alpha>=0, alpha <= c, y.T@alpha == 0] 
    prob = cp.Problem(obj, const)

    min_train_error=1
    y_pred=np.zeros(len(x_test))

    for c.value in C:       
        prob.solve()
        alpha1 = alpha.value

        # alpha1[np.abs(alpha1)<r]=0 #remove small values

        ## Compute b
        idx = np.where((alpha1 >= 1e-6) & (alpha1 <= c.value), True, False)
        if len(idx) == 0:
            raise ValueError 
        
        b = np.mean(y[idx]-rbf_kernel_matrix(x[idx], x, sigma, alpha1, y))

        ## Compute train error
        y_pred_train = np.sign(rbf_kernel_matrix(x, x, sigma, alpha1, y)+ b)
        train_error = np.mean(y_pred_train != y) #train error

        if min_train_error>train_error: #update minimum train error
            min_train_error=train_error

            ## Compute test prediction
            f_x = rbf_kernel_matrix(x_test, x, sigma, alpha1, y) + b # Shape: (n_x_test, 1)
            # print(f_x.shape)
            y_pred=np.array([i if f_x[k]>0 else j for k in range(len(x_test))]) #prediction for test data

    return min_train_error,y_pred

In [623]:
def logis(i,x1:pd.DataFrame,j,x2:pd.DataFrame,x_test:pd.DataFrame,r:float)-> int|np.ndarray:
    x=pd.concat([x1,x2]) #combine two classes
    y=pd.DataFrame(np.zeros((x1.shape[0]+x2.shape[0],1)))
    y.iloc[:x1.shape[0]]=1 #label for class 1
    y.iloc[x1.shape[0]:]=-1 #label for class 2

    x=x.values #convert to np
    y=y.values
    x_test=x_test.values

    w=cp.Variable((n,1))
    b=cp.Variable()

    obj=cp.Minimize(cp.sum(cp.logistic(-cp.multiply(y,x@w-b)))/x.shape[0])
    const=[]

    prob=cp.Problem(obj,const)
    
    prob.solve()

    w1=w.value #pd to np
    b1=b.value

    w1[np.abs(w1)<=r]=0 #remove small values
    # print((w1==0).T) #number of zero values

    train_error=np.mean(y!=np.sign(x@w1-b1)) #error rate
    # print(train_error)

    pred=np.array([i if (x_test@w1-b1)[k]>0 else j for k in range(len(x_test))]) #prediction for test data

    return train_error,pred


In [624]:
def solver(i,x1:pd.DataFrame,j,x2:pd.DataFrame,x_test:pd.DataFrame,g:np.ndarray,r:float,sigma:float,C:np.ndarray,T:str):
    if T=='svm':
        return svm(i,x1,j,x2,x_test,g,r)
    elif T=='logistic':
        return logis(i,x1,j,x2,x_test,r)
    elif T=='svm+kernel':
        return svm_kernel(i,x1,j,x2,x_test,r,sigma,C)

In [625]:
x_train_1=classify(y1_train) #split the data into different families
x_train_2=classify(y2_train) #split the data into different species
# print(x_train_1[0].shape)
# print(x_train_2[0].shape)

In [626]:

y1_test=mapp(y1_train,y1_test) #mapping family to 0,1,2,3
y2_test=mapp(y2_train,y2_test) #mapping species to 0,1,2,3
# print(y2_test)


C:\Users\86136\AppData\Local\Temp\ipykernel_17776\2238513263.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y_test.replace(mapping) #mapping y_test to 0,1,2,3
C:\Users\86136\AppData\Local\Temp\ipykernel_17776\2238513263.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return y_test.replace(mapping) #mapping y_test to 0,1,2,3


In [627]:
def confusion_matrix(y_true,y_pred):
    n=len(np.unique(y_true))
    cm=np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            cm[i,j]=np.sum((y_true==i)&(y_pred==j))
    return cm

In [628]:
def one_vs_one(x_train:pd.DataFrame,x_test:pd.DataFrame,y_test:pd.DataFrame,g:np.array,R:np.array,sigma:float,C:np.ndarray,T:str):

    m=len(x_train) #number of classes
    nm=(m*(m-1)//2) #number of models

    train_error=np.ones(nm)
    pred=np.zeros((nm,len(x_test))) #predict raw result for test data
    result=np.zeros((len(y_test),1)) #predict result for test data
    # test_error=np.zeros(len(g)) 

    t=0 #index for train_error and pred
    for i in range(m): 
        for j in range(i+1,m): #solve C(m,2) models
            for r in R:
                temp=solver(i,x_train[i],j,x_train[j],x_test,g,r,sigma,C,T)
                # print(i,j,r,temp[0])
                if train_error[t]>temp[0]:
                    train_error[t],pred[t]=temp #minimize train error over r
            t=t+1

    #analyze pred           
    for idx,col in enumerate(pred.T):
        counts=np.bincount(col.astype(int)) #count the number of each class
        result[idx]=np.argmax(counts) #choose the class with the most counts as the prediction

    test_error=np.sum(result!=y_test.values)/len(y_test) #error rate for test data
    mean_train_error=np.mean(train_error)

    print("test_error:",test_error)
    print("mean_train_error:",mean_train_error)
    print("confusion matrix:\n",confusion_matrix(y_test.values,result))


In [ ]:
def one_vs_rest(x_train:pd.DataFrame,x_test:pd.DataFrame,y_test:pd.DataFrame,g:np.array,R:np.array,sigma:float,C:np.ndarray,T:str):

    m=len(x_train) #number of classes
    
    train_error=np.ones(m)
    pred=np.zeros((m,len(x_test)))  #predict raw result for test data
    result=np.zeros((len(y1_test),1)) #predict result for test data
    # test_error=np.zeros(len(g)) 

    for i in range(m): #solve m models
        x_train_rest=pd.concat([x_train[j] for j in range(m) if j!=i]) #combine all other classes
        for r in R:
            temp=solver(i,x_train[i],-1,x_train_rest,x_test,g,r,sigma,C,T)
            if train_error[i]>temp[0]:
                train_error[i],pred[i]=temp #minimize train error over r

    #analyze pred
    for idx,col in enumerate(pred.T):
        if (np.sum(col != -1)==1): #classification unambiguous
            result[idx]=np.max(col)
        else: #classification ambiguous
            result[idx]=-1 

    test_error=np.sum(result!=y1_test.values)/len(y1_test) #error rate for test data
    mean_train_error=np.mean(train_error)
            
    print("test_error:",test_error)
    print("mean_train_error:",mean_train_error)
    print("confusion matrix:\n",confusion_matrix(y_test.values,result))

In [615]:
def model(task,Type,class_tech,r):
    '''
    task='family' or 'species',\n
    Type='svm' or 'logistic' or 'svm+kernel',\n
    class_tech='one_vs_rest' or 'one_vs_one',\n
    r=post-processing parameter,
    '''
    if task=='family':
        x_train=x_train_1
        y_test=y1_test
    elif task=='species':
        x_train=x_train_2
        y_test=y2_test

    if class_tech=='one_vs_one':
        g=np.array([0.1,1,5,10,50,1e2,1e3,1e4,1e5]) #gamma for SVM
        sigma=1 #sigma for RBF kernel
        C=np.array([1e-3,1e-2,1e-1,1,1e1,1e2,1e3,1e5,1e7])

        one_vs_one(x_train,x_test,y_test,g,r,sigma,C,Type)

    elif class_tech=='one_vs_rest':
        g=np.array([1e-2,0.1,1,5,10,50,1e2,1e3,1e4,1e5]) #gamma for SVM
        sigma=1 #sigma for RBF kernel
        C=np.array([1e-3,1e-2,1e-1,1,1e1,1e2,1e3,1e5,1e7])

        one_vs_rest(x_train,x_test,y_test,g,r,sigma,C,Type)
        


In [555]:

r=np.exp(np.linspace(-3,1,20))
model('family','svm','one_vs_one',r)


test_error: 0.04983388704318937
mean_train_error: 0.03775584232918908
confusion matrix:
 [[174.   0.   1.   0.]
 [  7.  85.   1.   3.]
 [  1.   1.  25.   0.]
 [  1.   0.   0.   2.]]


In [556]:


r=np.exp(np.linspace(-3,1,20))
model('species','svm','one_vs_one',r)


test_error: 0.06312292358803986
mean_train_error: 0.03847990192023274
confusion matrix:
 [[136.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  46.   1.   0.   0.   1.   0.   0.   0.   0.]
 [  0.   0.  27.   0.   1.   0.   1.   0.   0.   0.]
 [  0.   1.   1.  21.   0.   0.   0.   0.   0.   0.]
 [  0.   1.   0.   0.  25.   1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1.   9.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  10.   0.   0.   0.]
 [  0.   0.   1.   0.   0.   0.   0.   6.   2.   0.]
 [  0.   0.   0.   0.   0.   0.   1.   0.   2.   0.]
 [  0.   4.   0.   2.   0.   0.   0.   0.   0.   0.]]


In [557]:

r=np.exp(np.linspace(-3,1,20))
model('family','svm','one_vs_rest',r)


test_error: 0.09966777408637874
mean_train_error: 0.07832618025751073
confusion matrix: [[169.   1.   1.   0.]
 [  5.  83.   2.   0.]
 [  2.   1.  19.   0.]
 [  1.   2.   0.   0.]]


In [558]:
### ???
r=np.exp(np.linspace(-4,1,20))
model('species','svm','one_vs_rest',r)


test_error: 0.4053156146179402
mean_train_error: 0.018597997138769667
confusion matrix: [[135.   0.   0.   0.   0.   0.   0.   1.   0.   0.]
 [  0.  44.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  24.   0.   1.   1.   0.   0.   0.   0.]
 [  0.   0.   1.  17.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  19.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   7.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   9.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   1.   0.   4.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   1.   0.   0.   0.]
 [  0.   0.   0.   1.   0.   0.   0.   0.   0.   0.]]


In [629]:
r=np.exp(np.linspace(-3,1,20))
# r=np.linspace(1.6,1.7,20)
model('family','logistic','one_vs_one',r) 


d:\Downloadssoftware\Anaconda3\envs\py3-optimize\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


test_error: 0.04983388704318937
mean_train_error: 0.007303077725612937
confusion matrix:
 [[174.   0.   1.   0.]
 [  7.  85.   1.   3.]
 [  1.   1.  25.   0.]
 [  1.   0.   0.   2.]]


In [560]:
r=np.exp(np.linspace(-4,3,20))
# r=np.linspace(1.6,1.7,20)
model('species','logistic','one_vs_one',r) 


test_error: 0.046511627906976744
mean_train_error: 0.0
confusion matrix:
 [[136.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  1.  47.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   1.  26.   0.   2.   0.   0.   0.   0.   0.]
 [  0.   2.   0.  20.   0.   0.   0.   0.   0.   1.]
 [  0.   0.   0.   0.  26.   1.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1.   9.   0.   0.   0.   0.]
 [  0.   1.   0.   0.   0.   0.   9.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   9.   0.   0.]
 [  0.   0.   1.   0.   0.   0.   0.   0.   2.   0.]
 [  0.   0.   0.   3.   0.   0.   0.   0.   0.   3.]]


In [561]:

r=np.exp(np.linspace(-3,1,20))
model('family','logistic','one_vs_rest',r)

test_error: 0.132890365448505
mean_train_error: 0.030042918454935626
confusion matrix: [[167.   1.   2.   1.]
 [  6.  75.   1.   0.]
 [  1.   0.  18.   0.]
 [  1.   0.   0.   1.]]


In [562]:
r=np.exp(np.linspace(-3,1,20))
model('species','logistic','one_vs_rest',r) 

test_error: 0.43521594684385384
mean_train_error: 0.006008583690987125
confusion matrix: [[134.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  36.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  21.   0.   2.   1.   0.   0.   0.   0.]
 [  0.   0.   0.  14.   0.   0.   0.   0.   1.   0.]
 [  0.   0.   0.   0.  24.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   9.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   9.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   1.   0.   5.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   2.   0.]
 [  0.   0.   0.   1.   0.   0.   0.   0.   0.   1.]]


In [563]:
r=np.exp(np.linspace(-3,1,20))
model('family','svm+kernel','one_vs_one',r)


test_error: 0.2558139534883721
mean_train_error: 0.1486828865641258
confusion matrix:
 [[175.   0.   0.   0.]
 [ 47.  49.   0.   0.]
 [ 26.   1.   0.   0.]
 [  1.   2.   0.   0.]]


In [564]:
model('species','svm+kernel','one_vs_one',r)


test_error: 0.31893687707641194
mean_train_error: 0.255844995431067
confusion matrix:
 [[133.   0.   3.   0.   0.   0.   0.   0.   0.   0.]
 [  0.  43.   5.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  29.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  23.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  27.   0.   0.   0.   0.   0.   0.   0.]
 [  1.   0.   9.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.  10.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   9.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   1.   2.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   6.   0.   0.   0.   0.   0.   0.   0.]]


In [565]:
model('family','svm+kernel','one_vs_rest',r)


test_error: 0.2757475083056478
mean_train_error: 0.22765047554139267
confusion matrix: [[175.   0.   0.   0.]
 [ 37.  43.   0.   0.]
 [ 26.   0.   0.   0.]
 [  1.   0.   0.   0.]]


In [566]:
model('species','svm+kernel','one_vs_rest',r) 


test_error: 0.5946843853820598
mean_train_error: 0.11937224852179999
confusion matrix: [[77.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 44.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 15.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


There are some problems on the logistic model with species data and the kernel SVM model. The test error is so high and there are maybe some overfitting. I will continue to debug it.

3  
Theoretically, kernel SVM and one-vs-one should perform the best. Because it extracts more features from original datasets and creates more precise models.  
It is more difficult to predict species, because species has more categories than family.

4  
The logistic model is the least computationally intensive and the kernel SVM model is the most.  
The one-vs-rest technique is the least computationally intensive and the one-vs-one is the most.  
The more the computation is, the higher the testing accuracy is.

5  
I have not tested it. But I set the parameter r to remove small values of the parameter(such as a and w etc.) in the models. If the value is removed, the feature is least important.  
The easier the model is and the fewer categories the datasets have , the easier I might interpret it.

6  
We need to minimize x times error_rate + y times spending time.  
Since I have not got correct error rates on some of the above models, I can not get the lowest cost model. But based on the current result, I would like to to choose logistic model with one-vs-one technique for small scale classification problem (only few categories)